In [6]:
import pandas as pd
import json 

topic_annotation_table = "../data/pilot19_claim_topics/webapp_topicannotation.csv"
claims_path = "../data/dataset/perspectrum_with_answers_v0.1.json"

df = pd.read_csv(topic_annotation_table)
claims = json.load(open(claims_path))

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8966 entries, 0 to 8965
Data columns (total 4 columns):
id          8966 non-null int64
author      8966 non-null object
claim_id    8966 non-null int64
topics      8966 non-null object
dtypes: int64(2), object(2)
memory usage: 280.3+ KB


In [8]:
# Find how many claims we didn't cover in crowdsourcing

claims_ids = [c["cId"] for c in claims]
covered_cids = set(df.claim_id.unique().tolist())

missed_cids = sorted([cid for cid in claims_ids if cid not in covered_cids])

the_bin = []

print(missed_cids)


[909]


In [15]:
# build a histogram of category counts
topic_hist = {}
all_topics = df.topics.unique().tolist()

all_claims = sorted(df.claim_id.unique())

for cid in all_claims:
    topic_hist[cid] = [0 for k in all_topics]
    
for idx, row in df.iterrows():
    topic_hist[row.claim_id][all_topics.index(row.topics)] += 1

out_path = "../data/pilot19_claim_topics/topic_hist.csv"

with open(out_path, 'w') as fout:
    fout.write("claim_id," + ",".join(all_topics)+"\n")
    
    for cid in all_claims:
        counts = topic_hist[cid]
        fout.write(str(cid) + "," + ",".join([str(x) for x in counts]))
        fout.write('\n')

In [23]:
agg_count_path = "../data/pilot19_claim_topics/aggreated_counts.csv"

df = pd.read_csv(agg_count_path)
df.info()

topic_data = []

for idx, row in df.iterrows():
    d = row.to_dict()
    topics = [key for key, val in d.items() if val == 1 and key not in ["claim_id", "total count of topics "]]
    topic_data.append({
        "claim_id" : int(row.claim_id),
        "topics" : topics
    })
    
out_path = '../data/dataset/topic_data.json'

json.dump(topic_data, open(out_path, 'w'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006 entries, 0 to 1005
Data columns (total 20 columns):
claim_id                     1006 non-null int64
world_international          1006 non-null int64
politics                     1006 non-null int64
environment                  1006 non-null int64
education                    1006 non-null int64
digital_freedom              1006 non-null int64
economy                      1006 non-null int64
science_and_technology       1006 non-null int64
law                          1006 non-null int64
culture                      1006 non-null int64
health_and_medicine          1006 non-null int64
philosophy                   1006 non-null int64
religion                     1006 non-null int64
society                      1006 non-null int64
freedom_of_speech            1006 non-null int64
sports_and_entertainments    1006 non-null int64
human_rights                 1006 non-null int64
gender                       1006 non-null int64
ethics     

In [24]:
claim_v2 = '../data/dataset/v0.2/perspectrum_with_answers_v0.2.json'

claims = json.load(open(claim_v2, 'r'))

topic_dict = {}

for c in topic_data:
    topic_dict[c["claim_id"]] = c["topics"]
    
for c in claims:
    cid = c["cId"]
    c['topics'] =  topic_dict[cid]
    
out_path = '../data/dataset/v0.2/perspectrum_with_answers_topic_v0.2.json'
json.dump(claims, open(out_path, 'w'))